In [0]:
# COMMENT OUT FOR KAGGLE

!mkdir ../input
!wget "https://ai-camp-content.s3.amazonaws.com/reviews_train.csv" -P ../input
!wget "https://ai-camp-content.s3.amazonaws.com/reviews_test.csv" -P ../input

mkdir: cannot create directory ‘../input’: File exists
--2019-08-29 22:26:31--  https://ai-camp-content.s3.amazonaws.com/reviews_train.csv
Resolving ai-camp-content.s3.amazonaws.com (ai-camp-content.s3.amazonaws.com)... 52.216.112.163
Connecting to ai-camp-content.s3.amazonaws.com (ai-camp-content.s3.amazonaws.com)|52.216.112.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 257994545 (246M) [text/csv]
Saving to: ‘../input/reviews_train.csv.1’

reviews_train.csv.1 100%[===================>] 246.04M  3.16MB/s    in 44s     

2019-08-29 22:27:16 (5.62 MB/s) - ‘../input/reviews_train.csv.1’ saved [257994545/257994545]

--2019-08-29 22:27:18--  https://ai-camp-content.s3.amazonaws.com/reviews_test.csv
Resolving ai-camp-content.s3.amazonaws.com (ai-camp-content.s3.amazonaws.com)... 52.216.16.152
Connecting to ai-camp-content.s3.amazonaws.com (ai-camp-content.s3.amazonaws.com)|52.216.16.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 

In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import seaborn as sns
import re

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['reviews_train.csv.1', 'reviews_test.csv', 'reviews_train.csv', 'reviews_test.csv.1']


In [0]:
# define constants

TRAIN_PATH = "../input/reviews_train.csv"
TEST_PATH = "../input/reviews_test.csv"

NUM_FEATURES = 1000
PADDED_LENGTH = 250

In [0]:
# load data

# column names: positive, review_title, review_body
train_df = pd.read_csv(TRAIN_PATH, sep=',', header=0)
train_df.head(10)

,positive,review_title,review_body
0,0,mens ultrasheer,This model may be ok for sedentary types but I...
1,1,Surprisingly delightful,This is a fast read filled with unexpected hum...
2,0,Works but not as advertised,I bought one of these chargers..the instructio...
3,0,Oh dear,I was excited to find a book ostensibly about ...
4,0,Incorrect disc!,I am a big JVC fan but I do not like this mode...
5,0,Incorrect Disc,I love the style of this but after a couple ye...
6,0,DVD menu select problems,I cannot scroll through a DVD menu that is set...
7,0,My 2 y/o grandson loves it!!,This movie with all of its animals really keep...
8,1,A Cookbook Every Baker Should Own,I found a copy of this cookbook at a local use...
9,0,good basic,The book is a basic how to book for using sour...


In [0]:
# load test data

# column names: id, review_title, review_body
test_df = pd.read_csv(TEST_PATH)
test_df['Body and Title'] = test_df['review_title']  + ' ' + test_df['review_body']
test_df.head()

,id,review_title,review_body,Body and Title
0,0,Also works on the 2002 RSX base model!,This sensor is supposed to work on 02 - 04 Acu...,Also works on the 2002 RSX base model! This se...
1,1,Rsx Type-S o2 sensor Denso,Amazon is an amazing site I do not know what I...,Rsx Type-S o2 sensor Denso Amazon is an amazin...
2,2,Warning Mac OS X users!,Ok product otherwise but hangs OS X Classic en...,Warning Mac OS X users! Ok product otherwise b...
3,3,Instant Immersion Mandarin Chinese Review,I find the format difficult to follow unless y...,Instant Immersion Mandarin Chinese Review I fi...
4,4,it was rather pointless....,This is the third book in the Petsitter Myster...,it was rather pointless.... This is the third ...


In [0]:
# Solution goes here

#Import the necesary libraries 
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Embedding, LSTM, Conv1D, MaxPooling1D, Activation, Flatten, SpatialDropout1D
from keras.optimizers import SGD
from tqdm import tqdm

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer

import seaborn as sns
import re
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier

import numpy as np

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn import metrics
from __future__ import print_function
import time

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

In [0]:
#Concatanate the body and title together for the review 
train_df['title and body'] = train_df['review_title'] + ' ' + train_df['review_body']

#Also need to drop Nan values 
train_df = train_df.dropna()

train_df.head()

,positive,review_title,review_body,title and body
0,0,mens ultrasheer,This model may be ok for sedentary types but I...,mens ultrasheer This model may be ok for seden...
1,1,Surprisingly delightful,This is a fast read filled with unexpected hum...,Surprisingly delightful This is a fast read fi...
2,0,Works but not as advertised,I bought one of these chargers..the instructio...,Works but not as advertised I bought one of th...
3,0,Oh dear,I was excited to find a book ostensibly about ...,Oh dear I was excited to find a book ostensibl...
4,0,Incorrect disc!,I am a big JVC fan but I do not like this mode...,Incorrect disc! I am a big JVC fan but I do no...


In [0]:
#Split the data 
x_train, x_test, y_train, y_test = train_test_split(train_df['title and body'], train_df['positive'], test_size=0.25)

In [0]:
#Pipeline
text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,3))),
                     ('tfidf', TfidfTransformer(use_idf=True)),
                     ('clf', SGDClassifier(alpha=2e-5, loss='epsilon_insensitive', class_weight='balanced'))])

In [0]:
#Train the model
text_clf.fit(x_train, y_train)

In [0]:
#Predictions 
prediction1 = text_clf.predict(x_test)
metrics.accuracy_score(y_test, prediction1)

0.8656382695029642

In [0]:
#Hyperparameterisation
parameters = {'clf__penalty': ('none', 'l1', 'l2', 'elasticnet'),
              'clf__fit_intercept': (True, False),
              'clf__max_iter': (1000,1500,2000),
              'clf__class_weight': (None, 'balanced')}

gs_clf = GridSearchCV(text_clf, parameters, n_jobs=1, verbose=10, cv=2)

#Create a new split to find optimal hyperparamters on smaller data for better computation times 
x0, hyp_x, y0, hyp_y = train_test_split(train_df['title and body'], train_df['positive'], test_size = 0.05)

# Train the grid search algorithm with the data and target 
gs_clf.fit(hyp_x,hyp_y)

# Display the results
print("Best score: {}".format(gs_clf.best_score_))
print("Paremters used: {}".format(gs_clf.best_params_))
print("Execution time: " + str((stop_time - start_time)) + ' ms')

Fitting 2 folds for each of 48 candidates, totalling 96 fits
[CV] clf__class_weight=None, clf__fit_intercept=True, clf__max_iter=1000, clf__penalty=none 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  clf__class_weight=None, clf__fit_intercept=True, clf__max_iter=1000, clf__penalty=none, score=0.774, total=  12.3s
[CV] clf__class_weight=None, clf__fit_intercept=True, clf__max_iter=1000, clf__penalty=none 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   12.3s remaining:    0.0s


[CV]  clf__class_weight=None, clf__fit_intercept=True, clf__max_iter=1000, clf__penalty=none, score=0.773, total=  12.2s
[CV] clf__class_weight=None, clf__fit_intercept=True, clf__max_iter=1000, clf__penalty=l1 


...

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  2.0min remaining:    0.0s


[CV]  clf__class_weight=None, clf__fit_intercept=True, clf__max_iter=1500, clf__penalty=none, score=0.771, total=  12.2s
[CV] clf__class_weight=None, clf__fit_intercept=True, clf__max_iter=1500, clf__penalty=l1 
[CV]  clf__class_weight=None, clf__fit_intercept=True, clf__max_iter=1500, clf__penalty=l1, score=0.825, total=  13.5s
[CV] clf__class_weight=None, clf__fit_intercept=True, clf__max_iter=1500, clf__penalty=l1 
[CV]  clf__class_weight=None, clf__fit_intercept=True, clf__max_iter=1500, clf__penalty=l1, score=0.830, total=  13.6s
[CV] clf__class_weight=None, clf__fit_intercept=True, clf__max_iter=1500, clf__penalty=l2 
[CV]  clf__class_weight=None, clf__fit_intercept=True, clf__max_iter=1500, clf__penalty=l2, score=0.838, total=  12.5s
[CV] clf__class_weight=None, clf__fit_intercept=True, clf__max_iter=1500, clf__penalty=l2 
[CV]  clf__class_weight=None, clf__fit_intercept=True, clf__max_iter=1500, clf__penalty=l2, score=0.840, total=  12.5s
[CV] clf__class_weight=None, clf__fit_i

[Parallel(n_jobs=1)]: Done  96 out of  96 | elapsed: 21.0min finished


Best score: 0.8398974358974359
Paremters used: {'clf__class_weight': 'balanced', 'clf__fit_intercept': True, 'clf__max_iter': 1000, 'clf__penalty': 'l2'}


NameError: ignored

In [0]:
#Generate the predictions for Kaggle 
prediction = text_clf.predict(test_df['Body and Title'])

In [0]:
results_df = pd.DataFrame()
results_df['id'] = test_df['id']
results_df['positive'] = prediction

results_df.head()

,id,positive
0,0,1
1,1,1
2,2,0
3,3,0
4,4,0


In [0]:
results_df.to_csv('results.csv', ',', header=True, index=False)